In [16]:
# # 路径规划  2025/01/21 by chenxinyi
# 
# import networkx as nx
# import numpy as np
# import pandas as pd
# import time
# import copy
# import sys
# sys.path.append('C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\2.重构车辆路径\车辆路径规划\function')
# import function.route_plan
# from function.route_plan import get_path_attr
# from function.route_plan import topsis
# from function.route_plan import k_shortest_paths_select  # 改进k*算法(当最短路径经过的路网节点数＞10时，才启动k*算法)
# 
# import warnings
# warnings.filterwarnings("ignore")
import networkx as nx
import numpy as np
import pandas as pd
import time
import copy
import sys
import os

# 将 function 的“父目录”加入路径中
sys.path.append(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\2.重构车辆路径\车辆路径规划')

# 打印调试信息确认
print("[当前目录]", os.getcwd())
print("[路径是否存在]", os.path.exists(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\2.重构车辆路径\车辆路径规划\function\route_plan.py'))
from function_new import route_plan
from function_new.route_plan import get_path_attr, topsis, k_shortest_paths_select


import warnings
warnings.filterwarnings("ignore")

[当前目录] C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\2.重构车辆路径\车辆路径规划
[路径是否存在] False


In [17]:
# 读取路网拓扑结构
G = nx.read_gml(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_roadnet')
pos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_pos.npy', allow_pickle=True).item()
pos_name = list(pos.keys())
pos_coord = list(pos.values())
realpos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_realpos.npy')
realpos_name = list(realpos[0])
bayonet_name = list(realpos[1])

In [19]:
# 读取单次出行轨迹信息

file = 'C:/01 毕业论文/7.论文代码和结果/案例分析/数据/路径重构结果/中心城区单次出行轨迹结果'
# 单次出行轨迹经过的路网节点
path_pos_file = np.load(file + '/' + 'path_pos.npy', allow_pickle=True)
path_pos = path_pos_file.tolist()
# 单次出行轨迹经过的对应卡口设备
path_bayonet_file = np.load(file + '/' + 'path_bayonet.npy', allow_pickle=True)
path_bayonet = path_bayonet_file.tolist()
# 经过每一卡口设备的时间
path_time_file = np.load(file + '/' + 'path_time.npy', allow_pickle=True)
path_time = path_time_file.tolist()
# 起始时间
path_start_time = list(np.load(file + '/' + 'path_start_time.npy', allow_pickle=True))
# 结束时间
path_stop_time = list(np.load(file + '/' + 'path_stop_time.npy', allow_pickle=True))
# 行程时间
path_travel = list(np.load(file + '/' + 'path_travel.npy'))
# 车辆号牌
path_cltmbh = list(np.load(file + '/' + 'path_cltmbh.npy'))
# 原始数据经过的卡口个数
path_bayonet_number = list(np.load(file + '/' + 'path_bayonet_number.npy'))

print('单次出行路径:', len(path_cltmbh), len(path_start_time), len(path_stop_time), len(path_travel), 
      len(path_pos), len(path_bayonet), len(path_time), len(path_bayonet_number))

单次出行路径: 409511 409511 409511 409511 409511 409511 409511 409511


# 路径规划

In [20]:
# 随机森林训练获取的路径决策属性权重
importances = [0.3535516, 0.24072694, 0.31523439, 0.09048707]  # feature: length, crossing_num, kind_index, duration_rate

# 新建数据
# ind = 0
# historically_optimal_path = [[], [], []]    # 最优决策后的历史路径规划结果
# Complete_path_tf = []  # 记录是否补全完整路径vc
# path_pred_travel = []  # 基于路径规划计算的总预估行程时间序列
# Complete_path_pos = copy.deepcopy(path_pos)  # 路径规划后的路网节点序列和

# 读取已经处理好的数据
save_path = 'C:/01 毕业论文/7.论文代码和结果/案例分析/数据/路径重构结果/全部处理结果'
Complete_path_pos = list(np.load(save_path + '/' + 'Complete_path_pos.npy', allow_pickle=True))
Complete_path_tf = list(np.load(save_path + '/' + 'Complete_path_tf.npy', allow_pickle=True))
path_pred_travel = list(np.load(save_path + '/' + 'path_pred_travel.npy'))
print('已经处理数据量:', len(Complete_path_tf))
ind = len(Complete_path_tf)
historically_optimal_path = np.load('historically_optimal_path_250120.npy', allow_pickle=True).tolist()
historically_optimal_path[0] = list(historically_optimal_path[0])    # ndarray to list
historically_optimal_path[1] = list(historically_optimal_path[1])
historically_optimal_path[2] = list(historically_optimal_path[2])

已经处理数据量: 409511


In [24]:
# 基于KSP*+TOPSIS算法选择最优路径

start = time.perf_counter()
iteration = len(path_pos)
# 查询每一相邻卡口（路网节点）之间的路径
for m in range(ind, iteration):
    flag = "get"
    loc = 0
    
    od_pred_travel = []  # 每一od之间的路径预估行程时间
    for n in range(0, (len(path_time[m]) - 1)):
        source_pos = Complete_path_pos[m][loc]
        target_pos = Complete_path_pos[m][loc + 1]
        test_od = source_pos + ':' + target_pos
        source_time = path_time[m][n]
        target_time = path_time[m][n + 1]
        real_duration = (target_time-source_time).total_seconds()
          
        # 优先在历史路径集合中查询
        try:
            historically_index = historically_optimal_path[0].index(test_od)
        except:
            historically_index = -1
            
        
        # 1. 如果当前od在历史路径集合中存在，则调用历史路径
        if historically_index != -1:
            
            bestpath = historically_optimal_path[1][historically_index]
            bestpath_duration = historically_optimal_path[2][historically_index]
            od_pred_travel.append(bestpath_duration)
            
            # print(f"od_id={n}  {source_pos} -> {target_pos}")
            # print(f"bestpath_duration: {round(bestpath_duration)}, real_duration: {real_duration}")
            
            # 1.1 如果最优路径的预估行程时间远大于实际行驶时间，则标记该出行轨迹
            # if abs(real_duration-bestpath_duration) > real_duration:
                # flag = loc
                # Complete_path_pos[m] = path_pos[m]
                # break
               
            # 在原始轨迹中插入最优路径
            loc = loc + 1
            for z in range(1, len(bestpath) - 1):
                Complete_path_pos[m].insert(loc, bestpath[z])
                loc += 1
                
        # 2. 如果当前od在历史路径集合中不存在，则重新路径规划
        else:
            
            # 基于改进ksp算法获取k条最短路径
            k_durations, k_paths = k_shortest_paths_select(G, source_pos, target_pos, 3, 'duration')
            
            # 2.1 如果两个卡口之间存在路径，则在两个卡口对应的路网节点之间插入中间节点
            if k_paths != []:
                
                # 2.1.1 如果仅有1条最短路径，则选择其为最优路径
                if len(k_paths) == 1:
                    bestpath = k_paths[0]
                    bestpath_duration = k_durations[0]
                # 否则，通过topsis方法选择最优路径
                else:
                    # 计算决策矩阵
                    decision_matrix = get_path_attr(G, k_durations, k_paths, source_time, target_time)
                    # 基于TOPSIS决策方法求解最优路径
                    No, bestpath = topsis(k_paths, decision_matrix, importances)
                    bestpath_duration = k_durations[No]
                
                od_pred_travel.append(bestpath_duration)
                # print(f"od_id={n}  {source_pos} -> {target_pos}")
                # print(f"bestpath_duration: {round(bestpath_duration)}, real_duration: {real_duration}")
                
                # 2.1.2 如果最优路径的预估行程时间远大于实际行驶时间，则标记该出行轨迹
                if abs(real_duration-bestpath_duration) > real_duration:
                    flag = loc
                    Complete_path_pos[m] = path_pos[m]
                    break
                
                # 在原始轨迹中插入最优路径
                loc = loc + 1
                for z in range(1, len(bestpath) - 1):
                    Complete_path_pos[m].insert(loc, bestpath[z])
                    loc += 1
                    
                # 存储历史最优路径规划结果
                historically_optimal_path[0].append(test_od)    # 存储起终点od: source_pos-target_pos
                historically_optimal_path[1].append(bestpath)   # 存储od间的最优路径
                historically_optimal_path[2].append(bestpath_duration)   # 存储od间的最优路径的预估行程时间
            
            # 2.2 若两节点间不存在路径，则标记该出行轨迹
            else:  
                flag = loc
                Complete_path_pos[m] = path_pos[m]
                break

    Complete_path_tf.append(flag)
    
    # 保存每一次出行的预估行程时间
    if flag == 'get':
        path_pred_travel.append(round(sum(od_pred_travel)))
    else:
        path_pred_travel.append(np.nan)
    
    # print(f"{m}  路径规划结果: {flag}  卡口记录数: {len(path_bayonet[m])}  处理后的节点数: {len(Complete_path_pos[m])}  估计行程时间: {path_pred_travel[m]}({path_travel[m]})")
    # 定期打印算法运行时间和保存结果
    if m in list(np.arange(5000, 570000, 5000)):
        end = time.perf_counter()
        opt = end - start
        # print(f'第{m}次单次出行 路径规划完成  运行时间(min): {round(opt/60)}')
        print(f"{m}  路径规划结果: {flag}  卡口记录数: {len(path_bayonet[m])}  路网节点数: {len(Complete_path_pos[m])}  估计行程时间: {path_pred_travel[m]} (vs. {path_travel[m]})")
        np.save("historically_optimal_path_250120.npy", historically_optimal_path)
        np.save(save_path + '/' + 'Complete_path_tf.npy', Complete_path_tf)
        np.save(save_path + '/' + 'Complete_path_pos.npy', Complete_path_pos)
        np.save(save_path + '/' + 'path_pred_travel.npy', path_pred_travel)
        
print(1)
end = time.perf_counter()
opt = end - start
print("CPU Time:", round(opt / 3600, 2), "h")

# 提取路径规划成功的路径索引
positions = [index for index, value in enumerate(Complete_path_tf) if value == 'get']
print("路径规划成功率: {:.2%}".format(len(positions) / len(Complete_path_tf)), len(positions))

1
CPU Time: 0.0 h
路径规划成功率: 86.28% 353341


In [25]:
# 保存最优路径决策结果和全部处理结果
np.save("historically_optimal_path_250120.npy", historically_optimal_path)
np.save(save_path + '/' + 'Complete_path_tf.npy', Complete_path_tf)
np.save(save_path + '/' + 'Complete_path_pos.npy', Complete_path_pos)
np.save(save_path + '/' + 'path_pred_travel.npy', path_pred_travel)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 14202) + inhomogeneous part.

In [26]:
# 统计路径规划后，每一次出行经过的路网节点数
path_pos_number = []
for i in range(0, len(Complete_path_pos)):
    num = len(Complete_path_pos[i])
    path_pos_number.append(num)
    
print('单次出行平均路网节点记录数:', round(np.mean(path_pos_number), 2))
print('中位数平均值:', np.median(path_pos_number), '最小记录数:', min(path_pos_number), '最大记录数:', max(path_pos_number))
np.save(save_path + '/' + 'path_pos_number.npy', path_pos_number)

单次出行平均路网节点记录数: 86.78
中位数平均值: 73.0 最小记录数: 2 最大记录数: 1051


In [27]:
idx = 195000
print(f'路径索引 = {idx}')
print('Complete_path_tf:', Complete_path_tf[idx])
print('path_bayonet_number:', path_bayonet_number[idx])
print('path_pos_number:', path_pos_number[idx])
print('path_travel:', path_travel[idx])
print('path_pred_travel:', path_pred_travel[idx])
print('path_pos:', path_pos[idx])
print('Complete_path_pos:', Complete_path_pos[idx])

路径索引 = 195000
Complete_path_tf: get
path_bayonet_number: 5
path_pos_number: 21
path_travel: 783.0
path_pred_travel: 318.0
path_pos: [np.str_('LINK-299952938-777269799-808067190-808067191'), np.str_('3-641137380'), np.str_('4-790339725'), np.str_('LINK-637103766-637103767-637103792-637103793'), np.str_('1-194936760')]
Complete_path_pos: [np.str_('LINK-299952938-777269799-808067190-808067191'), 'LINK-299952938-641137380-777269798', '1-641137380', '2-641137380', np.str_('3-641137380'), '4-641137380', 'LINK-637103792-641137380', 'LINK-207265970-637103792', 'LINK-198402537-207265970-790339725', np.str_('4-790339725'), '3-790339725', 'LINK-641137380-790339725', 'LINK-641137379-790339725', 'LINK-637103792-641137379', 'LINK-637103792-641137380', 'LINK-207265970-637103792', np.str_('LINK-637103766-637103767-637103792-637103793'), 'LINK-637103766-668847689-669114991-833491423', 'LINK-668847689-668875027-680312863-831630216', 'LINK-194936760-195098320-680312863-680312864', np.str_('1-194936760')]

In [28]:
print('数据维度检验:')
print(len(path_cltmbh), len(path_travel), len(path_pred_travel), len(path_start_time), len(path_stop_time), len(path_pos_number), \
      len(Complete_path_tf), len(Complete_path_pos), len(path_pos_number))

数据维度检验:
409511 409511 409511 409511 409511 409511 409511 409511 409511
